In [34]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import lightgbm as lgb
from lightgbm.callback import early_stopping

In [4]:
PATH = "../../data"

# Load Data
def load_data():
    teams = pd.read_csv(f"{PATH}/MTeams.csv")
    seasons = pd.read_csv(f"{PATH}/MSeasons.csv") 
    seeds = pd.read_csv(f"{PATH}/MNCAATourneySeeds.csv") 
    regular_results = pd.read_csv(f"{PATH}/MRegularSeasonCompactResults.csv")
    tourney_results = pd.read_csv(f"{PATH}/MNCAATourneyCompactResults.csv") 
    detailed_results = pd.read_csv(f"{PATH}/MRegularSeasonDetailedResults.csv")  
    ordinal_ranks = pd.read_csv(f"{PATH}/MMasseyOrdinals.csv")  
    
    return teams, seasons, seeds, regular_results, tourney_results, detailed_results, massey_ordinals

teams, seasons, seeds, regular_results, tourney_results, detailed_results, ordinal_ranks = load_data()

display(teams.head())
display(seasons.head())
display(seeds.head())
display(regular_results.head())
display(tourney_results.head())
display(detailed_results.head())
display(ordinal_ranks.head())


,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025


,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/02/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [5]:
# INFO
display(teams.info())
display(seasons.info())
display(seeds.info())
display(regular_results.info())
display(tourney_results.info())
display(detailed_results.info())
display(ordinal_ranks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TeamID         380 non-null    int64 
 1   TeamName       380 non-null    object
 2   FirstD1Season  380 non-null    int64 
 3   LastD1Season   380 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 12.0+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Season   41 non-null     int64 
 1   DayZero  41 non-null     object
 2   RegionW  41 non-null     object
 3   RegionX  41 non-null     object
 4   RegionY  41 non-null     object
 5   RegionZ  41 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.1+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Season  2558 non-null   int64 
 1   Seed    2558 non-null   object
 2   TeamID  2558 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 60.1+ KB


...

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117748 entries, 0 to 117747
Data columns (total 34 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Season   117748 non-null  int64 
 1   DayNum   117748 non-null  int64 
 2   WTeamID  117748 non-null  int64 
 3   WScore   117748 non-null  int64 
 4   LTeamID  117748 non-null  int64 
 5   LScore   117748 non-null  int64 
 6   WLoc     117748 non-null  object
 7   NumOT    117748 non-null  int64 
 8   WFGM     117748 non-null  int64 
 9   WFGA     117748 non-null  int64 
 10  WFGM3    117748 non-null  int64 
 11  WFGA3    117748 non-null  int64 
 12  WFTM     117748 non-null  int64 
 13  WFTA     117748 non-null  int64 
 14  WOR      117748 non-null  int64 
 15  WDR      117748 non-null  int64 
 16  WAst     117748 non-null  int64 
 17  WTO      117748 non-null  int64 
 18  WStl     117748 non-null  int64 
 19  WBlk     117748 non-null  int64 
 20  WPF      117748 non-null  int64 
 21  LFGM     1

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5489117 entries, 0 to 5489116
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Season         int64 
 1   RankingDayNum  int64 
 2   SystemName     object
 3   TeamID         int64 
 4   OrdinalRank    int64 
dtypes: int64(4), object(1)
memory usage: 209.4+ MB


None

In [6]:
# Descriptive analysis
display(teams.describe())
display(seasons.describe())
display(seeds.describe())
display(regular_results.describe())
display(tourney_results.describe())
display(detailed_results.describe())
display(ordinal_ranks.describe())

,TeamID,FirstD1Season,LastD1Season
count,380.000000,380.000000,380.000000
mean,1290.500000,1989.618421,2023.952632
std,109.840794,9.755814,5.686894
min,1101.000000,1985.000000,1985.000000
25%,1195.750000,1985.000000,2025.000000
50%,1290.500000,1985.000000,2025.000000
75%,1385.250000,1986.250000,2025.000000
max,1480.000000,2025.000000,2025.000000


,Season
count,41.000000
mean,2005.000000
std,11.979149
min,1985.000000
25%,1995.000000
50%,2005.000000
75%,2015.000000
max,2025.000000


,Season,TeamID
count,2558.000000,2558.000000
mean,2004.376466,1290.900704
std,11.449682,103.447343
min,1985.000000,1101.000000
25%,1994.250000,1208.000000
50%,2004.000000,1285.000000
75%,2014.000000,1387.000000
max,2024.000000,1463.000000


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT
count,191796.000000,191796.000000,191796.000000,191796.000000,191796.000000,191796.000000,191796.000000
mean,2006.271257,73.602072,1287.599757,76.855664,1283.342286,64.771205,0.048937
std,11.623911,34.229230,104.920419,11.833224,105.102958,11.201711,0.258969
min,1985.000000,0.000000,1101.000000,34.000000,1101.000000,20.000000,0.000000
25%,1996.000000,45.000000,1199.000000,69.000000,1191.000000,57.000000,0.000000
50%,2007.000000,75.000000,1285.000000,76.000000,1281.000000,64.000000,0.000000
75%,2016.000000,103.000000,1380.000000,84.000000,1375.000000,72.000000,0.000000
max,2025.000000,132.000000,1480.000000,186.000000,1480.000000,150.000000,6.000000


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT
count,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000
mean,2004.374106,139.150913,1284.984114,76.867355,1291.419778,65.077442,0.066720
std,11.447616,4.218254,101.039491,11.768184,103.424835,11.043525,0.278191
min,1985.000000,134.000000,1101.000000,43.000000,1101.000000,29.000000,0.000000
25%,1994.250000,136.000000,1207.000000,69.000000,1209.000000,58.000000,0.000000
50%,2004.000000,137.000000,1276.000000,76.000000,1286.500000,64.000000,0.000000
75%,2014.000000,139.000000,1385.000000,84.000000,1387.000000,72.000000,0.000000
max,2024.000000,154.000000,1463.000000,149.000000,1463.000000,115.000000,3.000000


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,117748.000000,117748.000000,117748.00000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,...,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000,117748.000000
mean,2014.146355,70.294986,1288.25451,75.878936,1283.138830,63.888287,0.068689,26.401824,55.760242,7.347445,...,20.159790,12.073403,17.732454,10.461740,21.625650,11.409722,13.888907,6.004739,2.868185,19.305780
std,6.515929,35.772556,105.34750,10.998547,104.795432,10.848767,0.305098,4.680314,7.456374,3.119260,...,6.068136,5.344049,7.081056,4.221039,4.518197,3.724567,4.382700,2.745969,2.019050,4.553353
min,2003.000000,0.000000,1101.00000,34.000000,1101.000000,20.000000,0.000000,10.000000,26.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,4.000000
25%,2009.000000,40.000000,1199.00000,68.000000,1192.000000,57.000000,0.000000,23.000000,51.000000,5.000000,...,16.000000,8.000000,13.000000,7.000000,19.000000,9.000000,11.000000,4.000000,1.000000,16.000000
50%,2014.000000,73.000000,1287.00000,75.000000,1282.000000,64.000000,0.000000,26.000000,55.000000,7.000000,...,20.000000,12.000000,17.000000,10.000000,21.000000,11.000000,14.000000,6.000000,3.000000,19.000000
75%,2020.000000,101.000000,1381.00000,83.000000,1374.000000,71.000000,0.000000,29.000000,60.000000,9.000000,...,24.000000,15.000000,22.000000,13.000000,25.000000,14.000000,17.000000,8.000000,4.000000,22.000000
max,2025.000000,132.000000,1480.00000,149.000000,1480.000000,144.000000,6.000000,57.000000,103.000000,26.000000,...,59.000000,48.000000,65.000000,36.000000,49.000000,31.000000,41.000000,22.000000,18.000000,45.000000


,Season,RankingDayNum,TeamID,OrdinalRank
count,5.489117e+06,5.489117e+06,5.489117e+06,5.489117e+06
mean,2.015525e+03,8.474585e+01,1.285712e+03,1.737884e+02
std,6.081657e+00,3.303180e+01,1.051970e+02,1.015146e+02
min,2.003000e+03,0.000000e+00,1.101000e+03,1.000000e+00
25%,2.011000e+03,5.800000e+01,1.195000e+03,8.600000e+01
50%,2.016000e+03,8.600000e+01,1.284000e+03,1.730000e+02
75%,2.020000e+03,1.140000e+02,1.377000e+03,2.610000e+02
max,2.025000e+03,1.330000e+02,1.480000e+03,3.640000e+02


In [8]:
all_games = pd.concat([
    regular_results[['WTeamID', 'WScore', 'LScore']].rename(columns={'WTeamID': 'TeamID', 'WScore': 'PointsScored', 'LScore': 'PointsAllowed'}),
    regular_results[['LTeamID', 'LScore', 'WScore']].rename(columns={'LTeamID': 'TeamID', 'LScore': 'PointsScored', 'WScore': 'PointsAllowed'})
])

# Now you can group by TeamID to get overall averages, including wins & losses.
team_stats = all_games.groupby('TeamID').agg(
    AvgPointsScored=('PointsScored', 'mean'),
    AvgPointsAllowed=('PointsAllowed', 'mean'),
    TotalGames=('PointsScored', 'count')
)

wins = regular_results['WTeamID'].value_counts()
losses = regular_results['LTeamID'].value_counts()
team_stats['GamesWon'] = team_stats.index.map(wins).fillna(0).astype(int)
team_stats['GamesLost'] = team_stats.index.map(losses).fillna(0).astype(int)
team_stats['WinRatio'] = team_stats['GamesWon'] / team_stats['TotalGames']


In [9]:
team_stats.head()

,AvgPointsScored,AvgPointsAllowed,TotalGames,GamesWon,GamesLost,WinRatio
TeamID,,,,,,
1101,69.761905,71.285714,315,148,167,0.469841
1102,63.807240,67.726697,1105,395,710,0.357466
1103,71.453538,68.427962,1173,689,484,0.587383
1104,73.294678,68.310564,1259,795,464,0.631454
1105,65.927143,71.771429,700,244,456,0.348571


In [12]:
team_stats = team_stats.reset_index()

In [13]:
team_stats.columns

Index(['TeamID', 'AvgPointsScored', 'AvgPointsAllowed', 'TotalGames',
       'GamesWon', 'GamesLost', 'WinRatio'],
      dtype='object')

In [15]:
team_stats.head()

,TeamID,AvgPointsScored,AvgPointsAllowed,TotalGames,GamesWon,GamesLost,WinRatio
0,1101,69.761905,71.285714,315,148,167,0.469841
1,1102,63.807240,67.726697,1105,395,710,0.357466
2,1103,71.453538,68.427962,1173,689,484,0.587383
3,1104,73.294678,68.310564,1259,795,464,0.631454
4,1105,65.927143,71.771429,700,244,456,0.348571


In [16]:
detailed_results.sample(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
53700,2013,96,1409,74,1431,70,A,0,25,56,...,25,22,29,13,22,11,8,3,2,20
84763,2019,71,1261,83,1279,69,A,0,27,57,...,28,9,17,15,27,14,16,8,3,23
105835,2023,93,1371,84,1385,72,A,0,31,62,...,15,17,27,14,26,11,17,6,8,20
55433,2014,9,1332,107,1441,83,H,0,32,49,...,20,20,31,12,17,12,15,6,2,34
21588,2007,84,1105,68,1380,63,H,0,19,54,...,20,20,28,20,19,7,14,13,4,25
38720,2010,117,1112,71,1390,69,A,0,27,55,...,21,10,18,8,24,12,12,5,2,13
117369,2025,99,1103,105,1444,92,H,0,39,63,...,31,17,24,20,13,8,11,8,2,17
105140,2023,79,1329,72,1328,56,H,0,27,63,...,15,6,8,3,29,11,12,3,3,16
5352,2004,33,1461,92,1449,76,H,0,31,61,...,22,5,7,10,20,17,7,6,6,24
53558,2013,94,1375,65,1407,62,H,0,18,43,...,17,10,11,15,17,7,10,7,2,22


In [20]:
tourney_results.sample(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1547,2009,137,1338,72,1190,62,N,0
2380,2022,146,1314,69,1389,49,N,0
2059,2017,136,1235,84,1305,73,N,0
1308,2005,139,1301,65,1163,62,N,0
730,1996,138,1280,63,1343,41,N,0
271,1989,137,1231,99,1206,85,N,0
1168,2003,137,1462,71,1407,59,N,0
2278,2021,138,1242,93,1186,84,N,0
2507,2024,144,1181,54,1222,51,N,0
2198,2019,136,1277,76,1133,65,N,0


In [50]:
detailed_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [52]:
win_stats = detailed_results[['Season', 'WTeamID', 'WScore', 'WFGM', 'WFGA', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WStl', 'WBlk', 'WPF']]
lose_stats = detailed_results[['Season', 'LTeamID', 'LScore', 'LFGM', 'LFGA', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LStl', 'LBlk', 'LPF']]

win_stats = win_stats.rename(columns={'WTeamID': 'TeamID', 'WScore': 'Score', 'WFGM': 'FGM', 'WFGA': 'FGA', 'WFTM': 'FTM', 
                                      'WFTA': 'FTA', 'WOR': 'OR', 'WDR': 'DR', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF'})

lose_stats = lose_stats.rename(columns={'LTeamID': 'TeamID', 'LScore': 'Score', 'LFGM': 'FGM', 'LFGA': 'FGA', 'LFTM': 'FTM', 
                                        'LFTA': 'FTA', 'LOR': 'OR', 'LDR': 'DR', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF'})

combined_stats = pd.concat([win_stats, lose_stats])

# Grouping by TeamID and aggregate the stats
aggregated_stats = combined_stats.groupby(['Season', 'TeamID']).agg({
    'Score': 'sum', 
    'FGM': 'sum', 
    'FGA': 'sum', 
    'FTM': 'sum', 
    'FTA': 'sum', 
    'OR': 'sum', 
    'DR': 'sum', 
    'Stl': 'sum', 
    'Blk': 'sum', 
    'PF': 'sum'
}).reset_index()

In [53]:
aggregated_stats.head()

,Season,TeamID,Score,FGM,FGA,FTM,FTA,OR,DR,Stl,Blk,PF
0,2003,1102,1603,536,1114,312,479,117,471,167,50,525
1,2003,1103,2127,733,1508,514,698,264,538,196,63,536
2,2003,1104,1940,673,1601,416,586,380,670,185,106,505
3,2003,1105,1866,634,1602,401,568,351,601,242,54,526
4,2003,1106,1781,656,1548,298,461,344,668,234,88,509


In [64]:
# Aggregate wins and losses for each team
agg_wins = regular_results.groupby(['Season', 'WTeamID']).size().reset_index(name='wins')
agg_losses = regular_results.groupby(['Season', 'LTeamID']).size().reset_index(name='losses')

# Rename 'WTeamID' and 'LTeamID' to 'TeamID' for merging
agg_wins = agg_wins.rename(columns={'WTeamID': 'TeamID'})
agg_losses = agg_losses.rename(columns={'LTeamID': 'TeamID'})

# Debugging: Print first few rows of aggregated wins and losses
print("Aggregated Wins:")
print(agg_wins.head())
print("Aggregated Losses:")
print(agg_losses.head())

# Merge wins and losses 
agg_results = pd.merge(agg_wins, agg_losses, on=['Season', 'TeamID'], how='outer')

print("Merged Results (Before NaN Handling):")
print(agg_results.head())
# Fill NaN values with 0 for missing wins or losses
agg_results['wins'] = agg_results['wins'].fillna(0)
agg_results['losses'] = agg_results['losses'].fillna(0)
print("Merged Results (After NaN Handling):")
print(agg_results.head())

# Calculate total games (sum of wins and losses)
agg_results['total_games'] = agg_results['wins'] + agg_results['losses']

# Calculate win percentage (wins/total_games)
agg_results['win_percentage'] = agg_results['wins'] / agg_results['total_games']

# Debugging: Print results after calculating win percentage
print("Aggregated Results with Win Percentage:")
print(agg_results.head())

# Merging with ordinal ranks to add the team's rank to their season stats
final_aggregated_stats = pd.merge(
    agg_results,
    ordinal_ranks[['Season', 'TeamID', 'OrdinalRank']],  # Assuming you have this dataframe for ranks
    on=['Season', 'TeamID'],  # Merge by 'Season' and 'TeamID'
    how='left'
)

# Debugging: Print merged result with ordinal ranks
print("Final Aggregated Stats with Ordinal Ranks:")
print(final_aggregated_stats.head())

final_aggregated_stats = pd.merge(
    final_aggregated_stats,
    aggregated_stats,  
    on=['Season', 'TeamID'],  # Merge on 'Season' and 'TeamID'
    how='left'
)

print("Final Aggregated Stats with Team Stats:")
print(final_aggregated_stats.head())
print("Final Aggregated Stats:")
print(final_aggregated_stats.sample(10))


Aggregated Wins:
   Season  TeamID  wins
0    1985    1102     5
1    1985    1103     9
2    1985    1104    21
3    1985    1106    10
4    1985    1108    19
Aggregated Losses:
   Season  TeamID  losses
0    1985    1102      19
1    1985    1103      14
2    1985    1104       9
3    1985    1106      14
4    1985    1108       6
Merged Results (Before NaN Handling):
   Season  TeamID  wins  losses
0    1985    1102   5.0    19.0
1    1985    1103   9.0    14.0
2    1985    1104  21.0     9.0
3    1985    1106  10.0    14.0
4    1985    1108  19.0     6.0
Merged Results (After NaN Handling):
   Season  TeamID  wins  losses
0    1985    1102   5.0    19.0
1    1985    1103   9.0    14.0
2    1985    1104  21.0     9.0
3    1985    1106  10.0    14.0
4    1985    1108  19.0     6.0
Aggregated Results with Win Percentage:
   Season  TeamID  wins  losses  total_games  win_percentage
0    1985    1102   5.0    19.0         24.0        0.208333
1    1985    1103   9.0    14.0         23.

In [65]:
final_aggregated_stats.sample(20)

,Season,TeamID,wins,losses,total_games,win_percentage,OrdinalRank,Score,FGM,FGA,FTM,FTA,OR,DR,Stl,Blk,PF
3018302,2017,1250,19.0,12.0,31.0,0.612903,138.0,2339.0,828.0,1726.0,425.0,552.0,221.0,830.0,210.0,96.0,530.0
3550101,2019,1163,16.0,17.0,33.0,0.484848,82.0,2436.0,874.0,1959.0,436.0,641.0,381.0,790.0,200.0,145.0,643.0
2451584,2015,1275,12.0,19.0,31.0,0.387097,277.0,2054.0,713.0,1627.0,446.0,629.0,260.0,688.0,236.0,61.0,588.0
345676,2005,1290,13.0,15.0,28.0,0.464286,229.0,1875.0,690.0,1686.0,297.0,437.0,385.0,674.0,263.0,137.0,518.0
3330720,2018,1258,10.0,20.0,30.0,0.333333,252.0,2139.0,769.0,1753.0,406.0,568.0,362.0,599.0,212.0,71.0,596.0
354261,2005,1318,21.0,9.0,30.0,0.700000,127.0,2212.0,790.0,1802.0,398.0,559.0,314.0,731.0,215.0,194.0,493.0
4823009,2023,1332,19.0,14.0,33.0,0.575758,74.0,2332.0,842.0,1874.0,415.0,585.0,318.0,802.0,187.0,158.0,487.0
441517,2006,1182,3.0,24.0,27.0,0.111111,299.0,1786.0,650.0,1536.0,320.0,438.0,290.0,587.0,122.0,92.0,514.0
4468657,2022,1266,19.0,12.0,31.0,0.612903,46.0,2306.0,835.0,1842.0,369.0,494.0,186.0,789.0,244.0,158.0,541.0
3821299,2019,1461,7.0,24.0,31.0,0.225806,332.0,2040.0,672.0,1612.0,455.0,631.0,163.0,783.0,171.0,84.0,569.0


In [66]:
tourney_results.sample(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
93,1986,137,1424,74,1419,51,N,0
654,1995,137,1330,89,1437,81,N,3
2216,2019,137,1416,73,1433,58,N,0
168,1987,139,1207,82,1326,79,N,0
1000,2000,144,1409,80,1274,71,N,0
259,1989,136,1256,83,1247,74,N,0
340,1990,137,1345,75,1419,63,N,0
2257,2021,137,1196,75,1439,70,N,1
1072,2002,134,1373,81,1108,77,N,0
578,1994,136,1393,92,1218,78,N,0


In [70]:
team_wins = tourney_results.groupby(['Season', 'WTeamID']).size().reset_index(name='wins')
team_wins.head()  

,Season,WTeamID,wins
0,1985,1104,2
1,1985,1116,1
2,1985,1120,2
3,1985,1130,2
4,1985,1181,1


In [72]:
team_wins = team_wins.rename(columns={'wins': 'tournament_wins'})
team_wins.head()  

,Season,WTeamID,tournament_wins
0,1985,1104,2
1,1985,1116,1
2,1985,1120,2
3,1985,1130,2
4,1985,1181,1


In [73]:
final_aggregated_stats_with_wins = final_aggregated_stats.merge(
    team_wins[['Season', 'WTeamID', 'tournament_wins']], 
    how='left', 
    left_on=['Season', 'TeamID'], 
    right_on=['Season', 'WTeamID']
)

final_aggregated_stats_with_wins = final_aggregated_stats_with_wins.rename(columns={'wins_x': 'regular_season_wins'})

final_aggregated_stats_with_wins = final_aggregated_stats_with_wins.drop(columns=['WTeamID'])


In [75]:
final_aggregated_stats_with_wins.sample(20)

,Season,TeamID,wins,losses,total_games,win_percentage,OrdinalRank,Score,FGM,FGA,FTM,FTA,OR,DR,Stl,Blk,PF,tournament_wins
4845630,2023,1358,21.0,7.0,28.0,0.750000,29.0,1919.0,673.0,1533.0,355.0,514.0,248.0,644.0,216.0,57.0,524.0,NaN
1831644,2013,1219,15.0,13.0,28.0,0.535714,235.0,1958.0,730.0,1642.0,371.0,527.0,343.0,672.0,196.0,115.0,529.0,NaN
4920076,2023,1444,5.0,23.0,28.0,0.178571,311.0,1912.0,683.0,1623.0,327.0,452.0,280.0,618.0,127.0,59.0,489.0,NaN
1761635,2013,1129,19.0,10.0,29.0,0.655172,85.0,2061.0,719.0,1604.0,405.0,541.0,282.0,673.0,192.0,48.0,559.0,NaN
823687,2008,1401,23.0,10.0,33.0,0.696970,22.0,2347.0,833.0,1792.0,488.0,765.0,386.0,899.0,152.0,139.0,561.0,1.0
1004354,2009,1399,10.0,18.0,28.0,0.357143,254.0,2046.0,734.0,1684.0,387.0,561.0,357.0,648.0,236.0,150.0,597.0,NaN
1308987,2011,1177,7.0,24.0,31.0,0.225806,218.0,2104.0,766.0,1800.0,409.0,599.0,340.0,604.0,231.0,103.0,617.0,NaN
2619165,2016,1129,18.0,12.0,30.0,0.600000,100.0,2280.0,771.0,1724.0,499.0,705.0,281.0,734.0,189.0,61.0,594.0,NaN
1693056,2012,1393,31.0,2.0,33.0,0.939394,3.0,2461.0,910.0,1954.0,430.0,622.0,420.0,746.0,317.0,234.0,499.0,3.0
3635967,2019,1256,18.0,13.0,31.0,0.580645,157.0,2210.0,773.0,1793.0,425.0,644.0,336.0,774.0,176.0,108.0,508.0,NaN


In [76]:
final_aggregated_stats_with_wins.to_csv('../../submissions/season_stats_and_tourney_wins.csv', index=False)

# Feature Engineering

In [80]:
final_aggregated_stats_with_wins['tournament_wins'] = final_aggregated_stats_with_wins['tournament_wins'].fillna(0)

In [83]:
final_aggregated_stats_with_wins['FG%'] = final_aggregated_stats_with_wins['FGM'] / final_aggregated_stats_with_wins['FGA'] 

In [84]:
final_aggregated_stats_with_wins['DR_per_gm'] = final_aggregated_stats_with_wins['DR'] / final_aggregated_stats_with_wins['total_games'] 
final_aggregated_stats_with_wins['OR_per_gm'] = final_aggregated_stats_with_wins['OR'] / final_aggregated_stats_with_wins['total_games'] 
final_aggregated_stats_with_wins['Stl_per_gm'] = final_aggregated_stats_with_wins['Stl'] / final_aggregated_stats_with_wins['total_games']
final_aggregated_stats_with_wins['Blk_per_gm'] = final_aggregated_stats_with_wins['Blk'] / final_aggregated_stats_with_wins['total_games']

In [89]:
final_aggregated_stats_with_wins['FT%'] = final_aggregated_stats_with_wins['FTM'] / final_aggregated_stats_with_wins['FTA']

In [86]:
final_aggregated_stats_with_wins = final_aggregated_stats_with_wins.drop(columns=['DR', 'OR', 'Stl', 'Blk'])

In [92]:
final_aggregated_stats_with_wins.query('Season == 2023')[
    ['Season', 'TeamID', 'win_percentage', 'OrdinalRank', 'FG%', 'FT%', 'DR_per_gm', 'OR_per_gm', 'Stl_per_gm', 'Blk_per_gm', 'tournament_wins']
].sample(30)


,Season,TeamID,win_percentage,OrdinalRank,FG%,FT%,DR_per_gm,OR_per_gm,Stl_per_gm,Blk_per_gm,tournament_wins
4801569,2023,1307,0.656250,17.0,0.481618,0.743692,25.406250,7.812500,6.687500,3.718750,0.0
4922002,2023,1448,0.575758,69.0,0.459927,0.745514,24.333333,6.696970,6.030303,2.727273,0.0
4921949,2023,1448,0.575758,71.0,0.459927,0.745514,24.333333,6.696970,6.030303,2.727273,0.0
4882603,2023,1400,0.764706,12.0,0.468719,0.752322,23.147059,8.588235,8.058824,3.647059,3.0
4830268,2023,1340,0.344828,261.0,0.436406,0.731544,19.586207,6.931034,6.413793,3.172414,0.0
4762132,2023,1260,0.322581,258.0,0.462149,0.697446,21.677419,6.354839,5.645161,3.032258,0.0
4767887,2023,1267,0.741935,280.0,0.464181,0.697936,24.451613,10.225806,8.806452,5.225806,0.0
4638419,2023,1114,0.275862,338.0,0.443080,0.737303,22.793103,9.655172,6.793103,2.172414,0.0
4714431,2023,1205,0.448276,213.0,0.452426,0.625000,23.310345,7.896552,6.896552,4.517241,0.0
4882461,2023,1400,0.764706,5.0,0.468719,0.752322,23.147059,8.588235,8.058824,3.647059,3.0


In [90]:
final_aggregated_stats_with_wins.to_csv('../../submissions/season_stats_and_tourney_wins.csv', index=False)

In [99]:
final_aggregated_stats_with_wins.query('Season == 2023')[['TeamID', 'tournament_wins']] \
    .drop_duplicates() \
    .sort_values('tournament_wins', ascending=False)


,TeamID,tournament_wins
4676804,1163,6.0
4848173,1361,5.0
4774064,1274,4.0
4704176,1194,4.0
4719141,1211,3.0
...,...,...
4738669,1234,0.0
4737790,1233,0.0
4736911,1232,0.0
4735106,1230,0.0


In [108]:
final_aggregated_stats_with_wins_unique = final_aggregated_stats_with_wins.drop_duplicates(subset=['TeamID', 'Season'], keep='first')

In [114]:
final_aggregated_stats_with_wins_unique.query('Season == 2023')[
    ['Season', 'TeamID', 'win_percentage', 'OrdinalRank', 'FG%', 'FT%', 'DR_per_gm', 'OR_per_gm', 'Stl_per_gm', 'Blk_per_gm', 'tournament_wins']
].sort_values('tournament_wins', ascending=False)

,Season,TeamID,win_percentage,OrdinalRank,FG%,FT%,DR_per_gm,OR_per_gm,Stl_per_gm,Blk_per_gm,tournament_wins
4676804,2023,1163,0.757576,25.0,0.459869,0.754414,24.151515,11.787879,6.545455,4.878788,6.0
4848173,2023,1361,0.812500,17.0,0.441451,0.735577,23.437500,9.218750,7.125000,3.718750,5.0
4774064,2023,1274,0.781250,38.0,0.485849,0.776224,22.906250,8.968750,7.343750,3.281250,4.0
4704176,2023,1194,0.906250,140.0,0.468275,0.717584,26.062500,9.125000,6.500000,2.531250,4.0
4719141,2023,1211,0.843750,2.0,0.522160,0.701937,24.406250,8.937500,7.250000,3.250000,3.0
...,...,...,...,...,...,...,...,...,...,...,...
4738669,2023,1234,0.593750,24.0,0.454455,0.738532,23.187500,10.406250,6.625000,3.437500,0.0
4737790,2023,1233,0.794118,131.0,0.460798,0.731343,23.117647,10.323529,7.470588,5.558824,0.0
4736911,2023,1232,0.636364,211.0,0.481227,0.767828,25.454545,5.757576,6.363636,1.909091,0.0
4735106,2023,1230,0.321429,335.0,0.429847,0.749042,19.785714,7.535714,8.142857,2.821429,0.0


In [113]:
final_aggregated_stats_with_wins_unique.to_csv('../../submissions/unique_season_stats_and_tourney_wins.csv', index=False)

In [ ]:
final